In [ ]:
import time
import re
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# URL Target
URL_PRODUK = "https://shopee.co.id/POCO-F7-12-512GB-Snapdragon-8s-Gen-4-90W-Charge-i.51925611.44303086179"

In [ ]:
# Chrome Options
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1280,720")

# Inisialisasi Driver
driver = uc.Chrome(options=options, use_subprocess=True)

# Buka halaman utama untuk suntik cookie
driver.get("https://shopee.co.id/")

# Cookieee
spc_ec_value = ".eFZmeG4wN1I4MGRvWWZoabURT/bxbzmHW3Yg0asuFxvlgRTc053u05Wt1ax5ah68eXXtVHG4LHKPWzvRwUlb+COdVCO+6+D7IcOEK1iZuXl6BQsTj4IhzwI0EyernUwV6W8tG91Z7DbqdvX2wIinivWLsVTjkOGN5rZD5q05wsjUGMg3dLymfOdXHPbP39jgptIXqbZ8xDvHof7+UeAsAwb2jz4gTmVbgHPrf/023z3ByBis9b1xaoofLuX+B9+6LxqwkIG+qRDZwG1SEs7GIg=="

cookie = {
    'name': 'SPC_EC',
    'value': spc_ec_value,
    'domain': '.shopee.co.id'
}
driver.add_cookie(cookie)

# Navigasi ke halaman produk
driver.get(URL_PRODUK)
print("Driver terinisialisasi dan cookie berhasil dimuat.")

Driver terinisialisasi dan cookie berhasil dimuat.


In [ ]:
all_reviews_data = []
halaman_ke = 1
max_halaman = 218

# Ekstrak Item ID untuk penamaan file
match = re.search(r'i\.(\d+)\.(\d+)', driver.current_url)
item_id = match.groups()[1] if match else "unknown_item"

while halaman_ke <= max_halaman:
    print(f" Memproses halaman {halaman_ke}...")
    
    try:
        # Tunggu elemen ulasan dimuat
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='shopee-product-comment-list']/div")))
        time.sleep(2) 

        review_elements = driver.find_elements(By.XPATH, "//div[@class='shopee-product-comment-list']/div")
        
        # Iterasi setiap ulasan di halaman aktif
        for review_element in review_elements:
            # Ekstrak komponen ulasan
            author = review_element.find_element(By.XPATH, ".//*[@class='InK5kS']").text
            comment = review_element.find_element(By.XPATH, ".//div[@class='meQyXP']").text
            
            # Rating Bintang
            stars_container = review_element.find_element(By.XPATH, ".//div[@class='rGdC5O']")
            stars = len(stars_container.find_elements(By.XPATH, "./svg"))

            # Meta Info (Waktu & Variasi)
            meta_info = review_element.find_element(By.XPATH, ".//div[@class='XYk98l']").text
            review_time = meta_info.split('|')[0].strip()
            variation = meta_info.split('Variasi: ')[1].strip() if 'Variasi: ' in meta_info else ""
            
            # Template Tags
            tags = [tag.text for tag in review_element.find_elements(By.XPATH, ".//span[@class='K5v3lN']")]
            
            all_reviews_data.append({
                'author': author, 
                'rating': stars, 
                'comment': comment, 
                'variation': variation,
                'review_time': review_time,
                'template_tags': ", ".join(tags)
            })

        # Logika Next Page
        try:
            next_button_xpath = "//button[contains(@class, 'shopee-icon-button--right') and not(@aria-disabled='true')]"
            next_button = driver.find_element(By.XPATH, next_button_xpath)
            driver.execute_script("arguments[0].click();", next_button)
            halaman_ke += 1
        except NoSuchElementException:
            print("Halaman terakhir dicapai.")
            break
            
    except TimeoutException:
        print(f"Timeout pada halaman {halaman_ke}. Menghentikan proses.")
        break

# Export Data ke CSV
if all_reviews_data:
    df = pd.DataFrame(all_reviews_data)
    nama_file = f'ulasan_shopee_{item_id}_{len(all_reviews_data)}.csv'
    df.to_csv(nama_file, index=False)
    print(f"Data berhasil disimpan: {nama_file}")
else:
    print("Tidak ada data yang diekstrak.")

[INFO] Memproses halaman 1...
[INFO] Memproses halaman 2...
[INFO] Halaman terakhir dicapai.
[INFO] Data berhasil disimpan: ulasan_shopee_44303086179_6.csv


In [ ]:
print("Memulai Operasi 'Police Lineup' di Halaman 2...")

wait = WebDriverWait(driver, 15)
review_list_container = wait.until(
    EC.presence_of_element_located((By.CLASS_NAME, "shopee-product-comment-list"))
)
time.sleep(2)

review_elements = driver.find_elements(By.XPATH, "//div[@class='shopee-product-comment-list']/div")

# Loop dan cetak blueprint untuk setiap ulasan
for i, review_element in enumerate(review_elements):
    print(f"\n--- MULAI CETAK BIRU ULASAN #{i+1} ---")

    blueprint_html = review_element.get_attribute('innerHTML')
    print(blueprint_html)

Memulai Operasi 'Police Lineup' di Halaman 2...
[ERROR] Gagal melakukan operasi 'Police Lineup': Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xa24103
	0xa24144
	0x82e71d
	0x87a03d
	0x87a41b
	0x8c17f2
	0x89c954
	0x8bee17
	0x89c706
	0x86da30
	0x86ed54
	0xc957b4
	0xc9098a
	0xa4c392
	0xa3c4c8
	0xa4324d
	0xa2c478
	0xa2c63c
	0xa167ca
	0x75dc5d49
	0x770cd6db
	0x770cd661

